In [1]:
from pyspark import SparkFiles

In [2]:
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Watches_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

watch_df =  spark.read.option('header', 'true').csv("file://"+SparkFiles.get("amazon_reviews_us_Watches_v1_00.tsv.gz"), inferSchema=True, sep='\t')
watch_df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
marketplace|customer_id| review_id|product_id|product_parent| product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase| review_headline| review_body| review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
 US| 3653882|R3O9SGZBVQBV76|B00FALQ1ZC| 937001370|Invicta Women's 1...| Watches| 5| 0| 0| N| Y| Five Stars|Absolutely love t...|2015-08-31 00:00:00|
 US| 14661224| RKH8BNC3L5DLF|B00D3RGO20| 484010722|Kenneth Cole New ...| Watches| 5| 0| 0| N| Y|I love thiswatch ...|I love this watch...|2015-08-31 00:00:00|
 US| 27324930|R2HLE8WKZSU3NL|B00DKYC7TK| 361166390|Ritche 22mm Black...| Watches| 2| 1| 1| N| Y| Two Stars| Scratches|2015-08-31 00:00:00|
 US| 7211452|R31U3UH5AZ42LL|B000EQS1JW| 958035625|Citizen Men's BM8...| Watches| 5| 0| 0| N| Y| Five Stars|It works well on ...|2015-08-31 00:00:00|
 US| 12733322|R2SV659OUJ945Y|B00A6GFD7S| 765328221|Orient ER27009B M...| Watches| 4| 0| 0| N| Y|Beautiful face, b...|Beautiful watch f...|2015-08-31 00:00:00|
 US| 6576411| RA51CP8TR5A2L|B00EYSOSE8| 230493695|Casio Men's GW-94...| Watches| 5| 0| 0| N| Y| No complaints|i love this watch...|2015-08-31 00:00:00|
 US| 11811565| RB2Q7DLDN6TH6|B00WM0QA3M| 549298279|Fossil Women's ES...| Watches| 5| 1| 1| N| Y| Five Stars|for my wife and s...|2015-08-31 00:00:00|
 US| 49401598|R2RHFJV0UYBK3Y|B00A4EYBR0| 844009113|INFANTRY Mens Nig...| Watches| 1| 1| 5| N| N|I was about to bu...|I was about to bu...|2015-08-31 00:00:00|
 US| 45925069|R2Z6JOQ94LFHEP|B00MAMPGGE| 263720892|G-Shock Men's Gre...| Watches| 5| 1| 2| N| Y| Perfect watch!|Watch is perfect....|2015-08-31 00:00:00|
 US| 44751341| RX27XIIWY5JPB|B004LBPB7Q| 124278407|Heiden Quad Watch...| Watches| 4| 0| 0| N| Y|Great quality and...|Great quality and...|2015-08-31 00:00:00|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
only showing top 10 rows

In [3]:
# Count 
watch_df.count()

Out[4]: 960872

In [4]:
from pyspark.sql.functions import to_date

# review_id DataFrame
review_id_df = watch_df.select(["review_id", "customer_id", "product_id", "product_parent", to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_df.show(10)

+--------------+-----------+----------+--------------+-----------+
 review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
R3O9SGZBVQBV76| 3653882|B00FALQ1ZC| 937001370| 2015-08-31|
 RKH8BNC3L5DLF| 14661224|B00D3RGO20| 484010722| 2015-08-31|
R2HLE8WKZSU3NL| 27324930|B00DKYC7TK| 361166390| 2015-08-31|
R31U3UH5AZ42LL| 7211452|B000EQS1JW| 958035625| 2015-08-31|
R2SV659OUJ945Y| 12733322|B00A6GFD7S| 765328221| 2015-08-31|
 RA51CP8TR5A2L| 6576411|B00EYSOSE8| 230493695| 2015-08-31|
 RB2Q7DLDN6TH6| 11811565|B00WM0QA3M| 549298279| 2015-08-31|
R2RHFJV0UYBK3Y| 49401598|B00A4EYBR0| 844009113| 2015-08-31|
R2Z6JOQ94LFHEP| 45925069|B00MAMPGGE| 263720892| 2015-08-31|
 RX27XIIWY5JPB| 44751341|B004LBPB7Q| 124278407| 2015-08-31|
+--------------+-----------+----------+--------------+-----------+
only showing top 10 rows

In [5]:
# Products DataFrame
products_df = watch_df.select(["product_id", "product_title"]).drop_duplicates()
products_df.show(10)

+----------+--------------------+
product_id| product_title|
+----------+--------------------+
B00EVX7V1I|Game Time Women's...|
B009S4DODY|XOXO Women's XO55...|
B00LBKXQRW|Anne Klein Women'...|
B0009P679Y|Invicta Men's 993...|
B00DHF30RU|M&c Women's | Cla...|
B00NIDA43Y|GuTe Classic Skel...|
B008EQDDPQ|Nautica Men's N13...|
B004VRBZ66|Timex Men's T2N63...|
B009BEO81I| Fossil Riley|
B008B39MTI|XOXO Women's XO55...|
+----------+--------------------+
only showing top 10 rows

In [6]:
# Customers DataFrame
customers_df = watch_df.groupby("customer_id").agg({"customer_id": "count"}).withColumnRenamed("count(customer_id)", "customer_count")
customers_df.show(10)

+-----------+--------------+
customer_id|customer_count|
+-----------+--------------+
 1567510| 1|
 19502021| 1|
 12819130| 1|
 35329257| 2|
 108460| 1|
 5453476| 1|
 29913055| 1|
 30717305| 1|
 1570030| 1|
 19032020| 1|
+-----------+--------------+
only showing top 10 rows

In [7]:
# Vine DataFrame
vine_df = watch_df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_df.show(10)

+--------------+-----------+-------------+-----------+----+
 review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
R3O9SGZBVQBV76| 5| 0| 0| N|
 RKH8BNC3L5DLF| 5| 0| 0| N|
R2HLE8WKZSU3NL| 2| 1| 1| N|
R31U3UH5AZ42LL| 5| 0| 0| N|
R2SV659OUJ945Y| 4| 0| 0| N|
 RA51CP8TR5A2L| 5| 0| 0| N|
 RB2Q7DLDN6TH6| 5| 1| 1| N|
R2RHFJV0UYBK3Y| 1| 1| 5| N|
R2Z6JOQ94LFHEP| 5| 1| 2| N|
 RX27XIIWY5JPB| 4| 0| 0| N|
+--------------+-----------+-------------+-----------+----+
only showing top 10 rows

In [8]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://database-1.cwoll1t89gv9.us-east-2.rds.amazonaws.com:5432/postgres"
config = {"user":"root", 
          "password": "200233032p", 
          "driver":"org.postgresql.Driver"}

In [9]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [10]:
# Write products_df to table in RDS
products_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [11]:
# Write customers_df to table in RDS
customers_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [12]:
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)